## NOTEBOOK: Profile a ResNet18 Model

This tutorial validates, compiles, and profiles a ResNet18 model for inference on Envise using the Idiom Software Stack.

Run this Jupyter notebook on an environment that has a GPU instance. 

The model will traverse through the following stages in the developer flow:

**Validate the model**
    
    The operator coverage tool is invoked at this stage and checks for supported and unsupported ONXX operators in the model. 

**Compile the model** 
    
    The compile() Idiom API is invoked at this stage and the ONNX model is compiled for Envise.

**Profile the model** 
    
    The profile() Idiom API is invoked at this stage and the model is executed at runtime in an Envise-simulated environment for performance metrics.

**SYSTEM COMPONENT MINIMUM REQUIREMENTS**

* CPU: Any X86-64 architecture with 4 cores
* RAM: 64 GB memory
* GPU: One Nvidia 2080

#### Set up Imports

In [11]:
import idiom
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import itertools

#### Convert file to ONNX
Specify the `batch_size` and use the variable where ever the batch size is needed.

In [12]:
import torchvision.models as models
model = models.resnet18(pretrained=False, num_classes=10)
model.load_state_dict(torch.load('/idiom-eap/examples/00-getting-started/resnet-18-imagewoof2-320.pth'))
model.eval()
onnx_file_name = 'rn18.onnx'
batch_size  = 2
# Input to the model
x = torch.randn(batch_size, 3, 320, 320, requires_grad=True)

torch.onnx.export(model,                  # model being run
                  x,                      # model input (or a tuple for multiple inputs)
                  onnx_file_name,         # where to save the model (can be a file or file-like object)
                  verbose=False,
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'],) # the model's output names


#### Validate Model

The model needs to get validated for Operator Coverage. Here the ONNX model is scanned and you get an output that shows a list of supported and unsupported operations by the compiler.

The ONNX file that is being validated is: `resnet-18-imagewoof2-320.onnx`

The ``idiom.cc.onnx.check_op_cov`` API command invokes the Operator Coverage functionality. It accepts one argument: an ONNX model.
 

In [13]:
from idiom.cc.onnx import check_op_cov
check_op_cov(onnx_file_name)

2022-08-15 17:16:39,282 - check-op-cov - INFO - check-op-cov v0.5.0
Date and time: August 15, 2022 17:16:39
Source model path: /idiom-eap/examples/00-getting-started/tutorials/resnet18/perf/rn18.onnx
2022-08-15 17:16:39,283 - check-op-cov - INFO - Output files will be saved in /idiom-eap/examples/00-getting-started/tutorials/resnet18/perf/rn18_opcov
2022-08-15 17:16:39,284 - check-op-cov - INFO - Running operator coverage tool...
2022-08-15 17:16:39,609 - check-op-cov - INFO - Finished running operator coverage tool.
2022-08-15 17:16:39,614 - check-op-cov - INFO - General messages from the compiler:
ONNX opset version 9
ONNX IR version 7
ONNX producer "pytorch" version 1.10
ONNX model version 0

2022-08-15 17:16:39,614 - check-op-cov - INFO - 49/49 operators from 7 op types passed. All ops are supported!
2022-08-15 17:16:39,616 - check-op-cov - INFO - Op types summary:
Op Type            Total          Supported      Unsupported    Failed         
--------------------------------------

Above, you can see that the model is fully supported. Thus, it is suitable for compilation. 

#### Compile

The ``idiom.cc.onnx.compile`` API command invokes the Idiom Compiler, where an ONNX model is compiled for Envise. It accepts mainly two arguments:

* **output_directory** 

    Directory where the output files will get stored after compilation.

* **onnx_file_path**
    
    Path to the ONNX model.

In [14]:
from idiom.cc.onnx import compile
compiled_model_path = 'compiled-onnx'

In [15]:
print('Starting compiling...')
idiom.cc.onnx.compile(compiled_model_path, onnx_file_name, batch_size=batch_size)
print('Done compiling')

Starting compiling...
Done compiling


#### Get Dataset

Get the ImageWoof dataset

In [6]:
# !wget https://s3.amazonaws.com/fast-ai-imageclas/imagewoof2-320.tgz

In [7]:
# !tar xf imagewoof2-320.tgz  # silent
# !echo "Done"

In [8]:
!ls -l imagewoof2-320  # check you have the train and validation folders

total 1168
-rw-r--r--  1 auro auro 1186099 Feb  6  2021 noisy_imagewoof.csv
drwxr-xr-x 12 auro auro    4096 Mar  7  2019 train
drwxr-xr-x 12 auro auro    4096 Mar  7  2019 val


#### Load Dataset

Load the dataset and initialize the Dataloader features.

In [16]:
def load_input_data(dataset_path: str):
    print('Loading input data')

    loader = torch.utils.data.DataLoader(
        datasets.ImageFolder(dataset_path, 
                             transforms.Compose([
                                 transforms.Resize((320, 320)),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])),
        batch_size=batch_size, 
        shuffle=False,
        num_workers=1, 
        pin_memory=False)

    data = [{'input': x[0].numpy()} for x in itertools.islice(loader, 1)]

    return data


#### Profile Model

The ``idiom.runtime.profile`` API command invokes the profiler. It measures the model’s performance metrics such as Inferences Per Second (IPS) and latency of your model for Envise by profiling the execution of the model at runtime. 

It accepts three arguments:

* **Compiled Model Directory**

    The Compiled Model Directory where the compilation output resides.

* **Input data**

    A sequence of dictionaries containing model inputs. 

* **Batch size**

    The number of samples within a batch. This value is used to compute performance metrics.
    
    Note that we are setting the batch size to **1** in this tutorial. 

In [17]:
from idiom.runtime import profile
dataset_path = 'imagewoof2-320'
print('Starting loading & profiling...this may take a few seconds')
input_data = load_input_data(dataset_path)
compiled_model_dir = 'compiled-onnx'
idiom.runtime.profile(compiled_model_dir, input_data, detailed_report=True)

Starting loading & profiling...this may take a few seconds
imagewoof2-320
Loading input data
(2, 3, 320, 320)
Profiling compiled-onnx
    Loading model
    Profiling model execution
        Running batch 1 of 1

Performance Report

Source model path: compiled-onnx
Batch size: 2
Number of Envises: 0.5

+------------------------------------+-------------------------+----------------------+
|         Measurement Scope          |   Inferences per Second |   Batch Latency (ms) |
+====================================+=========================+======================+
|  System Performance (CPU Compute,  |                      53 |                38.04 |
| Envise Compute, and Data Transfer) |                         |                      |
+------------------------------------+-------------------------+----------------------+
|  Envise Compute and Data Transfer  |                    2228 |                 0.9  |
+------------------------------------+-------------------------+-----------------

#### Conclusion

This tutorial shows how to validate, compile, and profile a ``ResNet18`` model, and measure its performance metrics for Envise-behavior.
